# **Quantitative Value Strategy**

In volatile and uncertain market times, intrinsic value investing takes on less of a risk than momentum investing, which is a stronger strategy in a bull market. 



**"Value investing"** means investing in the stocks that are the cheapest relative to common measures of business value (like earnings or assets).

Value investing was opularized by investors like Warren Buffet, Seth Klarman, and Benjamin Graham.

For this project, we are going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

##### **Library Imports**

In [76]:
import numpy as np
import pandas as pd 
import xlsxwriter
import requests
from scipy import stats
import math

##### **Importing Our List of Stocks & API Token**

We will need to import our list of stocks and our API token before proceeding. We will make sure the .csv file is still in your working directory and import it with the following command:

**Documentation Page for Public API:** https://iexcloud.io/docs/api/

1. We need is a **base URL** for the API: a URL that will start every HTTP request. (You have to add which specific endpoints that you want to retrieve from the API. Most APIs only expose certain data through each endpoint, which makes things faster if you only need to retrieve certain data. Ex. If you have the Google Maps API, you might only want the name of the location, so you would send coordinates and get back its name. You would not get back other information like population.)

    a. **Sandbox mode of API:** randomized data, free
        1. Find **Testing Sandbox** on webpage

**Acquiring an API Token**

- Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

- API tokens (and other sensitive information) should be stored in a secrets.py file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

- Download your secrets.py file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [77]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secret import IEX_CLOUD_API_TOKEN

##### **Making Our First API Call**
- Making our version of our value screener
- We will start by building a simple value screener that ranks securities based on a single metric (price-to-earnings ratio)
    - price-earnings-ratio: price/yearly earnings forecast
        - price-earnings (peRatio) on IEXcloud
            - on Quote endpoint: look on JSON RESPONSE: shows 'latestPrice' and 'peRatio'
            - on Quote's HTTP REQUEST: GET /stock/{symbol}/quote/{field}, copy into notebook


In [78]:
symbol = 'aapl'

# api_url = https://cloud.iexapis.com
# 1. Add stable to have stable version of API (as opposed to bleeding-edge version with \
    # features that  are not full tested)
# 2. take '{field}' off the end to parse entire JSON response instead of a single metric
# 3. Make it an f'
# 4. Add authentication token: interpolate with ?token = (iex cloud api token)
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'

# Create API call using requests library
data = requests.get(api_url)

# print status code to show if HTTP request is succesful
print(data.status_code)
data = requests.get(api_url).json()
print(data)

200
{'avgTotalVolume': 60699540, 'calculationPrice': 'close', 'change': 1.41, 'changePercent': 0.0079, 'close': 179.8, 'closeSource': 'official', 'closeTime': 1697054400449, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 179.82, 'delayedPriceTime': 1697054392782, 'extendedChange': 0.56, 'extendedChangePercent': 0.00311, 'extendedPrice': 180.36, 'extendedPriceTime': 1697068790968, 'high': 179.85, 'highSource': '15 minute delayed price', 'highTime': 1697054399970, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 179.76, 'iexCloseTime': 1697054399962, 'iexLastUpdated': 1697054399962, 'iexMarketPercent': 0.016307741200844616, 'iexOpen': 178.2, 'iexOpenTime': 1697031000032, 'iexRealtimePrice': 179.76, 'iexRealtimeSize': 17, 'iexVolume': 775451, 'lastTradeTime': 1697054399962, 'latestPrice': 179.8, 'latestSource': 'Close', 'latestTime': 'October 11, 2023', 'latestUpdate': 1697054400449, 'latestVolume': 47551098, 'low': 177.6, 'lowSource': '15 

##### **Parsing Our API Call**
- This API call has the metric we need (peRatio)
- Ex. of how to parse the metric from our API call

In [79]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

30.22

##### **Executing A Batch API Call & Building Our DataFrame**
- Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

- We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [80]:

# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(list, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(list), n):
        yield list[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now to create a blank DataFrame and add our data to the data frame one-by-one

In [81]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:

    # create a url f' that queries Quote endpoint from earlier
    # search batch and find batch requests and copy and paste the 2nd example link address
    # change symbols into interpolated value symbols={symbol_string} (iterator of first for loop)
    # only need quote endpoint: take away news,chart, from quote,news,chart
    # take off: range=1m&last=5'
    batch_api_call_url = f'https://cloud.iexapis.com/v1/stock/market/batch?symbols={symbol_string},fb&types=quote&token={IEX_CLOUD_API_TOKEN}'

    # Execute and verify HTTP request
    data = requests.get(batch_api_call_url)  
    print(data.status_code)

    # .json() to turn into python dictionary
    data = requests.get(batch_api_call_url).json()


    for symbol in symbol_string.split(','):
        # Check if the symbol exists in the data dictionary
        if symbol in data:
            # Create a new DataFrame for the current symbol's data
            new_data = pd.DataFrame(
                [[symbol, 
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A']],
                columns=my_columns
            )
            # Concatenate the new data to the final DataFrame
            final_dataframe = pd.concat([final_dataframe, new_data], ignore_index=True)
        else:
            # Print a message if the symbol is not found in the data
            print(f"Symbol '{symbol}' not found in data.")

final_dataframe

200
Symbol 'ABC' not found in data.
Symbol 'ANTM' not found in data.
200
Symbol 'DISCA' not found in data.
Symbol 'FBHS' not found in data.
200
Symbol 'HFC' not found in data.
200
Symbol 'NLOK' not found in data.
Symbol 'RE' not found in data.
200
Symbol 'VIAC' not found in data.
Symbol 'WLTW' not found in data.
200


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,113.38,29.68,N/A
1,AAL,12.48,2.91,N/A
2,AAP,51.26,8.88,N/A
3,AAPL,179.80,30.22,N/A
4,ABBV,149.34,30.67,N/A
...,...,...,...,...
491,YUM,118.38,23.92,N/A
492,ZBH,105.35,43.9,N/A
493,ZBRA,217.16,17.39,N/A
494,ZION,35.65,3.71,N/A


##### **Removing Glamour Stocks**

The opposite of a "value stock" is a "glamour stock".

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [82]:
# Identifying top 50 peRatios

# sort df
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = False, inplace = True)

# remove stocks w peRatio < 0
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]

# Parse for top 50
final_dataframe = final_dataframe[:50]

# reset index
final_dataframe.reset_index(inplace = True) # inplace = True to modify original df

# drop index column
final_dataframe.drop('index', axis = 1, inplace = True) # axis = 1 to drop columns instead of rows
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,WYNN,94.73,1518.11,N/A
1,LVS,46.52,698.5,N/A
2,WELL,85.81,353.86,N/A
3,TAP,60.05,319.08,N/A
4,KSU,293.59,279.61,N/A
5,VTR,43.11,225.47,N/A
6,STE,225.00,189.08,N/A
7,CRM,206.85,130.09,N/A
8,NVDA,468.06,113.06,N/A
9,AMZN,131.83,103.8,N/A


##### **Calculating the Number of Shares to Buy**

We now need to calculate the number of shares we need to buy.

To do this, we will use the portfolio_input function that we created in our momentum project.

I have included this function below.

In [83]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the portfolio_input function to accept a portfolio_size variable from the user of this script.



In [84]:
portfolio_input()
print('Portfolio Size:', portfolio_size)

Portfolio Size: 100000


We can now use the global portfolio_size variable to calculate the number of shares that our strategy should purchase.

In [85]:
# Calculate the position size for each stock in the portfolio
position_size = float(portfolio_size) / len(final_dataframe.index)

# Loop through each stock in the final_dataframe
for row in final_dataframe.index:
    # Calculate the number of shares to buy for each stock
    # and round down to the nearest whole number using math.floor
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[row, 'Price'])

final_dataframe


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,WYNN,94.73,1518.11,21
1,LVS,46.52,698.5,42
2,WELL,85.81,353.86,23
3,TAP,60.05,319.08,33
4,KSU,293.59,279.61,6
5,VTR,43.11,225.47,46
6,STE,225.00,189.08,8
7,CRM,206.85,130.09,9
8,NVDA,468.06,113.06,4
9,AMZN,131.83,103.8,15


##### **Building a Better (and More Realistic) Value Strategy**
- Every valuation metric has certain flaws.

- For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

- Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

- Investors typically use a composite basket of valuation metrics to build robust quantitative value strategies. **In this section, we will filter for stocks with the lowest percentiles on the following metrics:**

    - Price-to-earnings ratio
    - Price-to-book ratio
    - Price-to-sales ratio
    - Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
    - Enterprise Value divided by Gross Profit (EV/GP)

             
    

   
- Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [86]:
symbol = 'AAPL'

# Not using a batch API call so we can query the IEX cloud API for multiple symbols/endpoints
# copy batch call from earlier
# change {symbol_string} to {symbol}
# add endpoints you need to find metrics. ex. advanced-stats
batch_api_call_url = f'https://cloud.iexapis.com/v1/stock/market/batch?symbols={symbol},fb&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'

data = requests.get(batch_api_call_url).json()

# go to IEX cloud docs to see where the data points are contained in the IEX cloud API
# To find metric:
    # 1. print(data['AAPL'])
    # 2. print(data['AAPL']['advanced-stats']) or whichever endpoint your metric is in
    # 3. print(data['AAPL']['advanced-stats']['priceToBook'])


# - Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# - Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']


# - Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']


# These are not provided explicitly by IEX cloud API
# Do division manually

# - Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# - Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit


We use the abbreviation rv often. It stands for **robust value**, which is what we'll call this sophisticated strategy moving forward.

In [87]:
# Define the column names for the DataFrame
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

# Create an empty DataFrame with the defined columns
rv_dataframe = pd.DataFrame(columns=rv_columns)

# Loop through each symbol string in the list of symbol_strings
for symbol_string in symbol_strings:
    # Create the API URL for batch requests
    batch_api_call_url = f'https://cloud.iexapis.com/v1/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    
    # Send a GET request to the API and parse the JSON response
    data = requests.get(batch_api_call_url).json()

    # Loop through each symbol in the symbol_string
    for symbol in symbol_string.split(','):
        # Check if the symbol is found in the data
        if symbol in data:
            # Extract relevant data from the JSON response
            enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
            ebitda = data[symbol]['advanced-stats']['EBITDA']
            gross_profit = data[symbol]['advanced-stats']['grossProfit']
        else:
            # Print a message if the symbol is not found in the data and continue to the next symbol
            print(f"Symbol '{symbol}' not found in data.")
            continue
        
        # Calculate the EV/EBITDA ratio, handling cases where ebitda is None or 0
        try:
            ev_to_ebitda = enterprise_value / ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        # Calculate the EV/GP ratio, handling cases where gross_profit is None or 0
        try:
            ev_to_gross_profit = enterprise_value / gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN

        # Check if the symbol is found in the data (again)
        if symbol in data:
            # Create a new DataFrame with the extracted data for the current symbol
            new_data = pd.DataFrame(
                [[symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',  # Placeholder for 'Number of Shares to Buy'
                data[symbol]['quote']['peRatio'],
                'N/A',  # Placeholder for 'PE Percentile'
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',  # Placeholder for 'PB Percentile'
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',  # Placeholder for 'PS Percentile'
                ev_to_ebitda,
                'N/A',  # Placeholder for 'EV/EBITDA Percentile'
                ev_to_gross_profit,
                'N/A',  # Placeholder for 'EV/GP Percentile'
                'N/A'   # Placeholder for 'RV Score'
                ]], 
                columns=rv_columns
            )
            
            # Concatenate the new DataFrame to the rv_dataframe
            rv_dataframe = pd.concat([rv_dataframe, new_data], ignore_index=True)
        else:
            # Continue to the next symbol if it's not found in the data
            continue

rv_dataframe


Symbol 'ABC' not found in data.
Symbol 'ANTM' not found in data.
Symbol 'DISCA' not found in data.
Symbol 'FBHS' not found in data.
Symbol 'HFC' not found in data.
Symbol 'NLOK' not found in data.
Symbol 'RE' not found in data.
Symbol 'VIAC' not found in data.
Symbol 'WLTW' not found in data.


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,113.38,N/A,29.68,N/A,5.96,N/A,4.74,N/A,20.417234,N/A,9.715669,N/A,N/A
1,AAL,12.48,N/A,2.91,N/A,-1.83,N/A,0.1514,N/A,4.422553,N/A,0.578425,N/A,N/A
2,AAP,51.26,N/A,8.88,N/A,1.14,N/A,0.2761,N/A,5.652404,N/A,0.961589,N/A,N/A
3,AAPL,179.80,N/A,30.22,N/A,46.27,N/A,7.26,N/A,22.877196,N/A,16.999500,N/A,N/A
4,ABBV,149.34,N/A,30.67,N/A,20.43,N/A,4.69,N/A,11.410605,N/A,8.160777,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,118.38,N/A,23.92,N/A,-4,N/A,4.82,N/A,19.024611,N/A,8.552914,N/A,N/A
492,ZBH,105.35,N/A,43.9,N/A,1.89,N/A,3.26,N/A,15.185004,N/A,5.587088,N/A,N/A
493,ZBRA,217.16,N/A,17.39,N/A,3.82,N/A,2.08,N/A,13.011182,N/A,5.293890,N/A,N/A
494,ZION,35.65,N/A,3.71,N/A,1.07,N/A,1.11,N/A,2.791232,N/A,1.162130,N/A,N/A


##### **Dealing With Missing Data in Our DataFrame**

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using.

You can use pandas' isnull method to identify missing data:

In [88]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
5,ABMD,381.020,N/A,65.69,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
27,ALXN,182.500,N/A,59.45,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
38,AON,327.320,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
86,CERN,94.920,N/A,47.94,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
116,CTL,11.000,N/A,9.73,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
119,CTXS,103.900,N/A,39.96,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
133,DISCK,24.420,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
162,ETFC,49.260,N/A,14.27,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
180,FISV,114.230,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
182,FLIR,57.340,N/A,32.03,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are **two main approaches**:

Drop missing data from the data set (pandas' dropna method is useful here)
Replace missing data with a new value (pandas' fillna method is useful here)
In this tutorial, we will replace missing data with the average non-NaN data point from that column.

**Approach #1:**

In [89]:
# Iterate through a list of the NUMERIC columns in the DataFrame
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    # Fill missing values (NaN) in the specified column with the mean of that column
    # This helps to replace missing data with the average value for that particular column
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned.

In [90]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


##### **Calculating Value Percentiles**
We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

Price-to-earnings ratio
Price-to-book ratio
Price-to-sales ratio
EV/EBITDA
EV/GP

Here's how we'll do this:

In [91]:
# Import the `percentileofscore` function from the `scipy.stats` module
from scipy.stats import percentileofscore as score

# Create a dictionary that maps specific metrics to their corresponding percentile columns
metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

# Loop through each metric in the dictionary
for metric in metrics.keys():
    # Loop through each row in the DataFrame
    for row in rv_dataframe.index:
        # Calculate the percentile for the specific metric in the current row
        percentile = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])
        # Assign the calculated percentile to the corresponding percentile column in the DataFrame
        rv_dataframe.loc[row, metrics[metric]] = percentile / 100

# Print each percentile score to verify that they were calculated correctly
for metric in metrics.values():
    print(rv_dataframe[metric])

# Print the entire DataFrame to see the percentile values added
rv_dataframe


0       0.72379
1      0.094758
2        0.1875
3      0.731855
4      0.743952
         ...   
491    0.600806
492    0.872984
493    0.429435
494    0.096774
495    0.826613
Name: PE Percentile, Length: 496, dtype: object
0      0.771169
1      0.060484
2      0.164315
3      0.975806
4      0.953629
         ...   
491    0.056452
492    0.336694
493    0.637097
494    0.153226
495    0.943548
Name: PB Percentile, Length: 496, dtype: object
0      0.780242
1      0.012097
2      0.034274
3      0.887097
4      0.775202
         ...   
491    0.784274
492    0.610887
493    0.462702
494     0.21875
495     0.94254
Name: PS Percentile, Length: 496, dtype: object
0      0.822581
1      0.074597
2      0.118952
3      0.864919
4      0.387097
         ...   
491    0.782258
492    0.580645
493    0.465726
494    0.024194
495    0.915323
Name: EV/EBITDA Percentile, Length: 496, dtype: object
0      0.762097
1      0.020161
2      0.058468
3      0.955645
4      0.657258
         ...   
4

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,113.38,N/A,29.68,0.72379,5.96,0.771169,4.7400,0.780242,20.417234,0.822581,9.715669,0.762097,N/A
1,AAL,12.48,N/A,2.91,0.094758,-1.83,0.060484,0.1514,0.012097,4.422553,0.074597,0.578425,0.020161,N/A
2,AAP,51.26,N/A,8.88,0.1875,1.14,0.164315,0.2761,0.034274,5.652404,0.118952,0.961589,0.058468,N/A
3,AAPL,179.80,N/A,30.22,0.731855,46.27,0.975806,7.2600,0.887097,22.877196,0.864919,16.999500,0.955645,N/A
4,ABBV,149.34,N/A,30.67,0.743952,20.43,0.953629,4.6900,0.775202,11.410605,0.387097,8.160777,0.657258,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,118.38,N/A,23.92,0.600806,-4.00,0.056452,4.8200,0.784274,19.024611,0.782258,8.552914,0.689516,N/A
492,ZBH,105.35,N/A,43.90,0.872984,1.89,0.336694,3.2600,0.610887,15.185004,0.580645,5.587088,0.427419,N/A
493,ZBRA,217.16,N/A,17.39,0.429435,3.82,0.637097,2.0800,0.462702,13.011182,0.465726,5.293890,0.375,N/A
494,ZION,35.65,N/A,3.71,0.096774,1.07,0.153226,1.1100,0.21875,2.791232,0.024194,1.162130,0.078629,N/A


##### **Calculating the RV Score**

We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [92]:
# Import the `mean` function from the `statistics` module
from statistics import mean

# Loop through each row in the DataFrame
for row in rv_dataframe.index:
    # Create an empty list to store percentile values for each metric
    value_percentiles = []
    
    # Loop through each metric in the metrics dictionary
    for metric in metrics.keys():
        # Append the percentile value for the current metric to the list
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    
    # Calculate the mean (average) of the percentile values for all metrics and assign it to the 'RV Score' column
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

# Print the updated DataFrame with the 'RV Score' column
rv_dataframe


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,113.38,N/A,29.68,0.72379,5.96,0.771169,4.7400,0.780242,20.417234,0.822581,9.715669,0.762097,0.771976
1,AAL,12.48,N/A,2.91,0.094758,-1.83,0.060484,0.1514,0.012097,4.422553,0.074597,0.578425,0.020161,0.052419
2,AAP,51.26,N/A,8.88,0.1875,1.14,0.164315,0.2761,0.034274,5.652404,0.118952,0.961589,0.058468,0.112702
3,AAPL,179.80,N/A,30.22,0.731855,46.27,0.975806,7.2600,0.887097,22.877196,0.864919,16.999500,0.955645,0.883065
4,ABBV,149.34,N/A,30.67,0.743952,20.43,0.953629,4.6900,0.775202,11.410605,0.387097,8.160777,0.657258,0.703427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,118.38,N/A,23.92,0.600806,-4.00,0.056452,4.8200,0.784274,19.024611,0.782258,8.552914,0.689516,0.582661
492,ZBH,105.35,N/A,43.90,0.872984,1.89,0.336694,3.2600,0.610887,15.185004,0.580645,5.587088,0.427419,0.565726
493,ZBRA,217.16,N/A,17.39,0.429435,3.82,0.637097,2.0800,0.462702,13.011182,0.465726,5.293890,0.375,0.473992
494,ZION,35.65,N/A,3.71,0.096774,1.07,0.153226,1.1100,0.21875,2.791232,0.024194,1.162130,0.078629,0.114315


##### **Selecting the 50 Best Value Stocks**
As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [93]:
# Sort the DataFrame by the 'RV Score' column in ascending order (lower scores first)
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)

# Select the top 50 rows (stocks) with the lowest RV Scores
rv_dataframe = rv_dataframe[:50]

# Reset the index of the DataFrame to start from 0 and drop the old index
rv_dataframe.reset_index(drop=True, inplace=True)

rv_dataframe


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.51,N/A,0.43,0.090726,0.0456,0.0625,0.0934,0.002016,0.270279,0.018145,0.094878,0.004032,0.035484
1,LNC,24.10,N/A,-1.36,0.080645,0.8347,0.094758,0.2624,0.028226,-1.376320,0.016129,0.262408,0.006048,0.045161
2,AAL,12.48,N/A,2.91,0.094758,-1.8300,0.060484,0.1514,0.012097,4.422553,0.074597,0.578425,0.020161,0.052419
3,GM,30.98,N/A,4.32,0.100806,0.6045,0.078629,0.2552,0.020161,1.159623,0.020161,0.832264,0.042339,0.052419
4,WBA,22.60,N/A,-5.89,0.064516,0.9172,0.114919,0.1419,0.010081,-5.959770,0.014113,1.120146,0.074597,0.055645
5,XRX,15.00,N/A,-8.82,0.050403,0.6960,0.086694,0.3289,0.046371,5.104743,0.092742,0.942723,0.052419,0.065726
6,C,41.53,N/A,6.61,0.131048,0.4280,0.068548,0.6089,0.100806,3.631483,0.046371,0.642493,0.022177,0.07379
7,KSS,18.98,N/A,-20.30,0.032258,0.5672,0.074597,0.1193,0.008065,7.525758,0.185484,1.084554,0.070565,0.074194
8,SYF,29.66,N/A,5.48,0.114919,0.9868,0.135081,0.6143,0.102823,3.442368,0.038306,0.804499,0.038306,0.085887
9,WFC,39.69,N/A,9.92,0.199597,0.4516,0.070565,0.7197,0.122984,2.901050,0.02621,0.754534,0.032258,0.090323


##### **Calculating the Number of Shares to Buy**

We'll use the portfolio_input function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [94]:
portfolio_input()

In [95]:
# Calculate the position size for each stock in the portfolio
# Position size is the total portfolio size divided by the number of stocks in the portfolio
position_size = float(portfolio_size) / len(rv_dataframe.index)

# Iterate through each row (stock) in the DataFrame
for row in rv_dataframe.index:
    # Calculate the number of shares to buy for each stock
    # Number of shares is the floor division of position size by the stock's price
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe.loc[row, 'Price'])

rv_dataframe


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.51,569,0.43,0.090726,0.0456,0.0625,0.0934,0.002016,0.270279,0.018145,0.094878,0.004032,0.035484
1,LNC,24.10,82,-1.36,0.080645,0.8347,0.094758,0.2624,0.028226,-1.376320,0.016129,0.262408,0.006048,0.045161
2,AAL,12.48,160,2.91,0.094758,-1.8300,0.060484,0.1514,0.012097,4.422553,0.074597,0.578425,0.020161,0.052419
3,GM,30.98,64,4.32,0.100806,0.6045,0.078629,0.2552,0.020161,1.159623,0.020161,0.832264,0.042339,0.052419
4,WBA,22.60,88,-5.89,0.064516,0.9172,0.114919,0.1419,0.010081,-5.959770,0.014113,1.120146,0.074597,0.055645
5,XRX,15.00,133,-8.82,0.050403,0.6960,0.086694,0.3289,0.046371,5.104743,0.092742,0.942723,0.052419,0.065726
6,C,41.53,48,6.61,0.131048,0.4280,0.068548,0.6089,0.100806,3.631483,0.046371,0.642493,0.022177,0.07379
7,KSS,18.98,105,-20.30,0.032258,0.5672,0.074597,0.1193,0.008065,7.525758,0.185484,1.084554,0.070565,0.074194
8,SYF,29.66,67,5.48,0.114919,0.9868,0.135081,0.6143,0.102823,3.442368,0.038306,0.804499,0.038306,0.085887
9,WFC,39.69,50,9.92,0.199597,0.4516,0.070565,0.7197,0.122984,2.901050,0.02621,0.754534,0.032258,0.090323


##### **Formatting Our Excel Output** 

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [96]:
# Create an ExcelWriter object to write the DataFrame to an Excel file
writer = pd.ExcelWriter('quantitative_value_strategy.xlsx', engine='xlsxwriter')

# Write the 'rv_dataframe' DataFrame to an Excel sheet named 'Quantitative Value Strategy'
# The 'index = False' argument ensures that the index is not included in the Excel file
rv_dataframe.to_excel(writer, sheet_name='Quantitative Value Strategy', index=False)



##### **Creating the Formats We'll Need For Our .xlsx File**
Formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

- String format for tickers
- $XX.XX format for stock prices
- $XX,XXX format for market capitalization
- Integer format for the number of shares to purchase
- Float formats with 1 decimal for each valuation metric


In [97]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [98]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

# 'column_formats' is a dictionary containing column formatting information
# It's defined earlier in the code

# Loop through each column in 'column_formats'
for column in column_formats.keys():
    # Set the column width for the specified column
    # 'writer.sheets['Quantitative Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])'
    # - 'f'{column}:{column}' is used to specify the column range (in this case, a single column)
    # - '25' is the width of the column
    # - 'column_formats[column][1]' is the format to be applied to the column
    writer.sheets['Quantitative Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    
    # Write the header label to the specified column
    # 'writer.sheets['Quantitative Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])'
    # - 'f'{column}1'' specifies the cell where the header label should be written
    # - 'column_formats[column][0]' is the label or text to be written in the cell
    # - 'column_formats[column][1]' is the format to be applied to the cell
    writer.sheets['Quantitative Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])


##### **Saving Our Excel Output**


In [99]:
writer.close()

In [100]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.51,569,0.43,0.090726,0.0456,0.0625,0.0934,0.002016,0.270279,0.018145,0.094878,0.004032,0.035484
1,LNC,24.10,82,-1.36,0.080645,0.8347,0.094758,0.2624,0.028226,-1.376320,0.016129,0.262408,0.006048,0.045161
2,AAL,12.48,160,2.91,0.094758,-1.8300,0.060484,0.1514,0.012097,4.422553,0.074597,0.578425,0.020161,0.052419
3,GM,30.98,64,4.32,0.100806,0.6045,0.078629,0.2552,0.020161,1.159623,0.020161,0.832264,0.042339,0.052419
4,WBA,22.60,88,-5.89,0.064516,0.9172,0.114919,0.1419,0.010081,-5.959770,0.014113,1.120146,0.074597,0.055645
5,XRX,15.00,133,-8.82,0.050403,0.6960,0.086694,0.3289,0.046371,5.104743,0.092742,0.942723,0.052419,0.065726
6,C,41.53,48,6.61,0.131048,0.4280,0.068548,0.6089,0.100806,3.631483,0.046371,0.642493,0.022177,0.07379
7,KSS,18.98,105,-20.30,0.032258,0.5672,0.074597,0.1193,0.008065,7.525758,0.185484,1.084554,0.070565,0.074194
8,SYF,29.66,67,5.48,0.114919,0.9868,0.135081,0.6143,0.102823,3.442368,0.038306,0.804499,0.038306,0.085887
9,WFC,39.69,50,9.92,0.199597,0.4516,0.070565,0.7197,0.122984,2.901050,0.02621,0.754534,0.032258,0.090323
